<a href="https://colab.research.google.com/github/ctawong/Face-Depixelizer/blob/master/Face_Depixelizer_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="black" size="+4">Try PULSE</font></b>

Given a low-resolution input image, Face Depixelizer searches the outputs of a generative model (here, StyleGAN) for high-resolution images that are perceptually realistic and downscale correctly.

<b><font color="black" size="+2">Based on:</font></b>

**GitHub repository**: [PULSE](https://github.com/adamian98/pulse)

Article: [PULSE: Self-Supervised Photo Upsampling via Latent Space Exploration of Generative Models](https://arxiv.org/abs/2003.03808)





In [ ]:
import torch
import torchvision
from pathlib import Path
import shutil
import os

if not Path("PULSE.py").exists():
  if Path("pulse").exists():
    %cd /content/pulse
  else:
    !git clone https://github.com/ctawong/pulse
    %cd /content/pulse
    !mkdir input/
    toPIL = torchvision.transforms.ToPILImage()
    toTensor = torchvision.transforms.ToTensor()
    from bicubic import BicubicDownSample
    D = BicubicDownSample(factor=1)

# remove previous content
if Path("realpics").exists():
  shutil.rmtree('realpics')
!mkdir realpics
if Path("input").exists():
  shutil.rmtree('input')
!mkdir input


from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from PULSE import PULSE
from google.colab import files
from bicubic import BicubicDownSample
from IPython import display
from IPython.display import display
from IPython.display import clear_output
import numpy as np
from drive import open_url
from mpl_toolkits.axes_grid1 import ImageGrid
import requests
%matplotlib inline

#@markdown <- Paste **photo url** below and click play button on left to start!

#@markdown ---
#@markdown ## Required settings:
Photo_URL = "" #@param {type:"string"}
r = requests.get(Photo_URL, allow_redirects=True)
ext=r.headers.get('content-type').split('/')[1]
realPicName = 'realpics/face.'+ext
open(realPicName, 'wb').write(r.content)

!python align_face.py
!mv input/face_0.png input/face.png

# face = Image.open(fn)
# face = face.resize((1024, 1024), Image.ANTIALIAS)
# face = face.convert('RGB')
# face_name = 'face.png'
# face.save(face_name)
# %cp $face_name /content/pulse/input/

# images = []
# imagesHR = []
# imagesHR.append(face)
# face = toPIL(D(toTensor(face).unsqueeze(0).cuda()).cpu().detach().clamp(0,1)[0])
# images.append(face)


#@markdown ---
#@markdown ## Advanced settings:
#@markdown ##### *If you want to make a more accurate result, then modify the following* **DEFAULT** *variables*:

allowed_error = 0.0035 #@param {type:"slider", min:0.001, max:0.006, step:0.0005}
number_of_steps = 150 #@param {type:"slider", min:100, max:1000, step:50}
seed = 100 #@param {type:"integer"}
noise_type = 'trainable'  #@param ['zero', 'fixed', 'trainable']
optimizer = 'adam'  #@param ['sgd', 'adam','sgdm', 'adamax']
learning_rate = 0.4 #@param {type:"slider", min:0, max:1, step:0.05}
learning_rate_schedule = 'linear1cycledrop'  #@param ['fixed', 'linear1cycle', 'linear1cycledrop']
clear_output()

seed = abs(seed)
!python run.py \
  -eps $allowed_error\
  -seed $seed \
  -noise_type $noise_type \
  -opt_name $optimizer \
  -learning_rate $learning_rate \
  -steps $number_of_steps \
  -lr_schedule $learning_rate_schedule

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(mpimg.imread('/content/pulse/input/face.png'))
ax1.set_title('Original')

if Path('/content/pulse/runs/face.png').exists():
  ax2.imshow(mpimg.imread('/content/pulse/runs/face.png'))
  ax2.set_title('Result')
else:
  print("Error: Try increasing allowed_error and/or number_of_steps")
plt.show()

In [ ]:
Path('/content/pulse/runs/face.png').exists()

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Download result</font></b>
try: files.download('/content/pulse/runs/face.png')
except: raise Exception("No result image")